In [3]:
text = "안녕하세요 👋 (hello in Korean!)"

these characters have a integer associated with them defined by unicode

In [5]:
[ord(x) for x in text]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

These unicode numbers has a very large vocabulary, which we don't require for training and thus is not desirable for us.
Thus we encode these into bytes (utf-8).
Thus it restricts the mapping from 0 to 255.

In [7]:
list(text.encode("utf-8"))

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

These are raw bytes which we will call tokens.

In [9]:
train_text = "We used the Falcon chess engine as a baseline for our experiments. Falcon is a grandmaster-level chess program, which has successfully participated in several World Computer Chess Championships (WCCCs); in particular, it won second place at the World Computer Speed Chess Championship in 2008. Falcon’s extensiveevaluation function consists of more than 100 parameters, and its implementation contains several thousands of lines of code.  Despite all the computational improvements mentioned earlier for DeepChess, and numerous other implementation improvements which result in substantial additional computational speedup, DeepChess is still four times slower than Falcon’s own evaluation function. Nevertheless, we incorporate DeepChess into Falcon, completely replacing the evaluation function of the program. To measure the performance of DeepChess, we conducted a series of matches against Falcon, and also against the chess program Crafty. Crafty has successfully participated in numerous WCCCs, and is a direct descendant of Cray Blitz, the WCCC winner of 1983 and 1986. It has been frequently used in the literature as a standard reference. Each of the matches of DeepChess vs. Falcon and Crafty consisted of 100 games under a time control of 30 minutes per game for each side. Table 2 provides the results. As can be seen, DeepChess is on a par with Falcon. Falcon uses a manually tuned evaluation function developed over nearly ten years, containing more than a hundred parameters which grasp many subtle chess features. And yet, without any chess knowledge whatsoever (not even basic knowledge as the rules of chess), our DeepChess method managed to reach a level which is on a par with the manually tuned evaluation function of Falcon. The results also show that DeepChess is over 60 Elo [7] stronger than Crafty, a program which has won two WCCCs and has been manually tuned for thirty years. DeepChess performs on a par with Falcon despite the fact that it is four times slower. We ran a separate experiment where we allowed DeepChess to use four times more time than Falcon (2 hours vs 30 minutes). Running 100 such matches, DeepChess resoundingly defeated Falcon with a result of 63.5 - 36.5, corresponding to a 96 Elo performance difference. This shows that DeepChess is  actually not on par with Falcon’s evaluation function, but is considerably superior to it. In order to utilize the full potential of this enhanced chess understanding, it is critical to decrease the runtime of the neural network in the inference mode."

In [10]:
tokens = train_text.encode("utf-8") #raw bytes
tokens = list(map(int, tokens)) #convert to a list of integers

print("length", len(train_text))
print("no. of tokens:", len(tokens))

length 2540
no. of tokens: 2546


In [11]:
def get_stats(ids):
    counts = {} #dictionary to store all the pairs
    for pair in zip(ids, ids[1:]): 
        counts[pair] = counts.get(pair, 0) + 1  # pair: tuple[str, str]
    return counts

stats = get_stats(tokens)

In [12]:
top_pair = max(stats, key=stats.get)
top_pair

(115, 32)

Byte Pair Encoding

In [14]:
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

Now we need to find a sweet spot for the number of vocabulary terms vs number of tokens in our training text.

In [16]:
vocab_size = 271 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (115, 32) into a new token 256
merging (101, 32) into a new token 257
merging (111, 110) into a new token 258
merging (116, 104) into a new token 259
merging (97, 108) into a new token 260
merging (101, 115) into a new token 261
merging (101, 114) into a new token 262
merging (116, 105) into a new token 263
merging (100, 32) into a new token 264
merging (97, 110) into a new token 265
merging (105, 110) into a new token 266
merging (258, 32) into a new token 267
merging (104, 261) into a new token 268
merging (101, 110) into a new token 269
merging (44, 32) into a new token 270


In [17]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 2546
ids length: 2022
compression ratio: 1.26X


In [44]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids, vocab):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

In [42]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

Now we have completed the training of the transformer.
We will start embedding.

In [25]:
import numpy as np

class Embed():
    def __init__(self, vocab_size, embed_dim, max_len):
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size
        self.maz_len = max_len

        # Token Embedding: Maps token ids to a vector (learnable weights)
        self.token_embed = np.random.randn(vocab_size, embed_dim) / np.sqrt(embed_dim)

        #Position Embedding: Gives the token a position to get the position importance (Learnable weights in GPT-2, fixed in Attention is all you need paper)
        self.pos_embed = np.random.randn(max_len, embed_dim) / np.sqrt(embed_dim)

    def forward(self, token_ids):
        """
        token_ids: (batch_size, seq_len), dtype: int
        returns: (batch_size, seq_len, embed_dim)
        """
        batch_size, seq_len = token_ids.shape

        # Lookup token embeddings
        tok_embeds = self.token_embed[token_ids]  # (B, L, D)

        # Broadcast position embeddings to batch
        pos_embeds = self.pos_embed[np.arange(seq_len)]  # (L, D)
        pos_embeds = np.broadcast_to(pos_embeds, (batch_size, seq_len, self.embed_dim))

        return tok_embeds + pos_embeds  # final input embeddings
        
        

Multi Head Attention

In [28]:
import numpy as np

class MultiHeadSelfAttention:
    def __init__(self, embed_dim=768, num_heads=12):
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = 1 / np.sqrt(self.head_dim)

        # Q, K, V projection weights: [D, D]
        self.W_q = np.random.randn(embed_dim, embed_dim) / np.sqrt(embed_dim)
        self.W_k = np.random.randn(embed_dim, embed_dim) / np.sqrt(embed_dim)
        self.W_v = np.random.randn(embed_dim, embed_dim) / np.sqrt(embed_dim)

        # Output projection: [D, D]
        self.W_o = np.random.randn(embed_dim, embed_dim) / np.sqrt(embed_dim)

    def forward(self, x):
        """
        x: (B, L, D)
        returns: (B, L, D)
        """
        B, L, D = x.shape
        H, d = self.num_heads, self.head_dim

        # Linear projections
        Q = x @ self.W_q   # (B, L, D)
        K = x @ self.W_k
        V = x @ self.W_v

        # Reshape for heads: (B, H, L, d)
        Q = Q.reshape(B, L, H, d).transpose(0, 2, 1, 3)
        K = K.reshape(B, L, H, d).transpose(0, 2, 1, 3)
        V = V.reshape(B, L, H, d).transpose(0, 2, 1, 3)

        # Scaled dot-product attention: (B, H, L, L)
        attn_scores = Q @ K.transpose(0, 1, 3, 2) * self.scale

        # Causal mask: prevent attending to future tokens
        mask = np.triu(np.ones((L, L)), k=1).astype(bool)  # (L, L)
        attn_scores[:, :, mask] = -1e10

        # Softmax over last axis (L)
        attn_weights = np.exp(attn_scores - np.max(attn_scores, axis=-1, keepdims=True))
        attn_weights /= np.sum(attn_weights, axis=-1, keepdims=True)

        # Weighted sum of V
        attn_output = attn_weights @ V  # (B, H, L, d)

        # Merge heads back: (B, L, D)
        attn_output = attn_output.transpose(0, 2, 1, 3).reshape(B, L, D)

        # Final linear projection
        output = attn_output @ self.W_o  # (B, L, D)
        return output


Feed Forward Network

In [31]:
class FeedForward:
    def __init__(self, embed_dim=768, hidden_dim=3072):
        self.W1 = np.random.randn(embed_dim, hidden_dim) / np.sqrt(embed_dim)
        self.b1 = np.zeros(hidden_dim)

        self.W2 = np.random.randn(hidden_dim, embed_dim) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(embed_dim)

    def gelu(self, x):
        # Approximate GELU activation (used in GPT-2)
        return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))

    def forward(self, x):
        """
        x: (B, L, D)
        returns: (B, L, D)
        """
        h = x @ self.W1 + self.b1  # (B, L, H)
        h = self.gelu(h)           # GELU activation
        out = h @ self.W2 + self.b2  # (B, L, D)
        return out


In [36]:
class OutputProjection:
    def __init__(self, embedding_matrix):
        self.embedding_matrix = embedding_matrix  # (V, D)

    def forward(self, hidden_states):
        # hidden_states: (B, L, D)
        return hidden_states @ self.embedding_matrix.T  # (B, L, V)

    def predict(self, hidden_states):
        logits = self.forward(hidden_states)  # (B, L, V)
        return np.argmax(logits, axis=-1)     # (B, L)


Now lets test the model.. The outputs will be gibberish as the weights are randomly assigned and no training is done


In [46]:
input_text = "Engineering students"
input_ids = encode(input_text)  # list of ints

# Convert to batch (1, L)
input_ids = np.array([input_ids])  # shape: (1, L)

vocab_size = 271
embed_dim = 768
max_len = 128

embedder = Embed(vocab_size, embed_dim, max_len)
attention = MultiHeadSelfAttention(embed_dim=embed_dim)
ffn = FeedForward(embed_dim=embed_dim, hidden_dim=3072)
output_proj = OutputProjection(embedder.token_embed)

# Step 1: Embedding
x = embedder.forward(input_ids)  # (1, L, D)

# Step 2: Self-attention
x = attention.forward(x)         # (1, L, D)

# Step 3: Feed Forward
x = ffn.forward(x)               # (1, L, D)

# Step 4: Output projection
logits = output_proj.forward(x)  # (1, L, V)

# Step 5: Predict token ids
pred_ids = np.argmax(logits, axis=-1)  # (1, L)

# Convert output token ids back to text
decoded_text = decode(pred_ids[0], vocab)
print("Predicted text:", decoded_text)


Predicted text: ���bbbbbbbbbbbbb
